<a href="https://colab.research.google.com/github/Linthevanrooij/FAIP/blob/main/Final_project/Text%2Bimage-generator/Image-generator/image_generator_colab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Image generation using diffusion model

### Import libraries

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import json


In [1]:
!pip install diffusers

import torch
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Image generation for each feeling

In [2]:
stage_01 = "Help me define a design brief for a modern chair. The target audience is young professionals looking for a minimalist, ergonomic chair for home offices. Include materials and aesthetics."
stage_02 = "Brainstorm chair design ideas that combine minimalist aesthetics with ergonomic functionality. List 5 ideas, each with a description of form, materials, and key features."
stage_03 = "I am building a moodboard for a modern chair design. Suggest themes, color palettes, and material combinations for a sleek, minimalist style."
stage_04 = "I have a rough idea for a chair design: a curved backrest with thin metal legs and a wooden seat. Suggest refinements to improve comfort and aesthetics."
stage_05 = "Summarize the final concept of my chair design, highlighting its ergonomic features, minimalist aesthetic, and materials used. Prepare this for a product pitch."
all_stages = [stage_01, stage_02, stage_03, stage_04, stage_05]

In [24]:
def extra_emotion(valence, arousal):
  emotion = f"""
  Take into account my current emotional state without explicitly mentioning it. Valence is ranging from negative to positive. Arousal is intensity of emotion. Both ranging from 1-9.
  Arousal: {arousal}
  Valence: {valence}
  """
  return emotion

In [15]:
def calculate_valence_arousal(v1_lower, v2_upper, a1_lower, a2_upper):
  valence = random.uniform(v1_lower, v2_upper)
  arousal = random.uniform(a1_lower, a2_upper)

  valence = round(valence, 2)
  arousal = round(arousal, 2)

  return valence, arousal

In [30]:
def make_images(stage, stage_num, arousal_p, valence_p, arousal_n, valence_n):
    prompt_baseline = stage
    prompt_postive = stage + " " + extra_emotion(valence_p, arousal_p)
    prompt_negative = stage + " " + extra_emotion(valence_n, arousal_n)

    image_baseline = pipe(prompt_baseline).images[0]
    image_positive = pipe(prompt_postive).images[0]
    image_negative = pipe(prompt_negative).images[0]

    imageFile_baseline = f'baseline_stage_{stage_num}.png'
    imageFile_positive = f'positive_stage_{stage_num}.png'
    imageFile_negative = f'negative_{stage_num}.png'

    image_baseline.save(imageFile_baseline)
    image_positive.save(imageFile_positive)
    image_negative.save(imageFile_negative)

In [32]:
from io import open_code
all_scores = {}
for i, stage in enumerate(all_stages):
  i += 1
  valence_pos, arousal_pos = calculate_valence_arousal(7,9, 1,9)
  valence_neg, arousal_neg = calculate_valence_arousal(1,3, 1,9)

  all_scores[f'stage_{i}'] = {
    'prompt': stage,
    'valence_pos': valence_pos,
    'arousal_pos': arousal_pos,
    'valence_neg': valence_neg,
    'arousal_neg': arousal_neg
  }

  make_images(stage, i, arousal_pos, valence_pos, arousal_neg, valence_neg)

with open('all_scores.json', 'w') as f:
  f.write(json.dumps(all_scores, indent=2))


  0%|          | 0/50 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['. 7 8 valence : 7. 1 1']


  0%|          | 0/50 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['. 3 5 valence : 1. 8 4']


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [': 7. 9 6']


  0%|          | 0/50 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['2. 8']


  0%|          | 0/50 [00:00<?, ?it/s]

### Load saved trained model

In [ ]:
input_dim = 2524
num_classes = 3
model = EEGConformer(input_dim, num_classes)

In [ ]:
PATH = 'model/Conformer_EEG.pth'
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()


EEGConformer(
  (conv1): Conv2d(1, 40, kernel_size=(1, 25), stride=(1, 1))
  (conv2): Conv2d(40, 40, kernel_size=(1, 2524), stride=(1, 1))
  (batchnorm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layernorm1): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
  (multiheadattention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=40, out_features=40, bias=True)
  )
  (layernorm2): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
  (feedworward_block): Sequential(
    (0): Linear(in_features=40, out_features=32, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=32, out_features=40, bias=True)
  )
  (fc1): Linear(in_features=40, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=3, bias=True)
)

### Define model

In [ ]:
class EEGConformer(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EEGConformer, self).__init__()

        # CNN
        self.conv1 = nn.Conv2d(1, 40, kernel_size=(1, 25), stride=(1, 1))
        self.conv2 = nn.Conv2d(40, 40, kernel_size=(1, input_dim), stride=(1, 1))
        self.batchnorm = nn.BatchNorm2d(40)

        # Transformer
        self.layernorm1 = nn.LayerNorm(40)
        self.multiheadattention = nn.MultiheadAttention(40, 1)
        self.layernorm2 = nn.LayerNorm(40)

        self.feedworward_block = nn.Sequential(
            nn.Linear(40, 32),
            nn.GELU(),
            nn.Dropout(p=0.1),
            nn.Linear(32, 40)
        )

        # MLP
        self.fc1 = nn.Linear(40, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        # CNN
        x = x.unsqueeze(1).unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.batchnorm(x)

        # Transformer
        x = x.squeeze(2).squeeze(2)
        x = self.layernorm1(x)
        attn_out = self.multiheadattention(x, x, x)
        x = x + nn.Dropout(0.1)(attn_out[0])
        x = self.layernorm2(x)
        x = self.feedworward_block(x)
        x = nn.Dropout(p=0.1)(x)

        # MLP
        x = self.fc1(x)
        x = F.elu(x)
        x = nn.Dropout(p=0.5)(x)
        x = self.fc2(x)
        x = F.elu(x)
        x = nn.Dropout(p=0.3)(x)
        x = self.fc3(x)

        return x

### Load data

In [ ]:
eeg_emotions_data = pd.read_csv('data/emotions.csv')
eeg_emotions_data.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


### Pre-processing

In [ ]:
X = eeg_emotions_data.iloc[1:2,:].drop(['label'], axis=1)
y = eeg_emotions_data['label']

from sklearn.preprocessing import LabelEncoder
labelencoder_emotions = LabelEncoder()
y = labelencoder_emotions.fit_transform(y)

from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)
type(X)
X.shape

(1, 2548)

### Prediction

In [ ]:
prediction = model(torch.from_numpy(X).float()).detach().numpy().argmax(axis=1)
prediction = labelencoder_emotions.inverse_transform(prediction)
prediction

array(['POSITIVE'], dtype=object)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
from diffusers import StableDiffusionPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

def generate_image(model, modelPath, dataPath, idx, prePrompt):

  model.load_state_dict(torch.load(modelPath, weights_only=True))
  model.eval()

  data = pd.read_csv(dataPath)

  X = data.iloc[idx,:].drop(['label'], axis=1)
  y = data['label']

  labelencoder = LabelEncoder()
  y = labelencoder.fit_transform(y)

  X = StandardScaler().fit_transform(X)

  prediction = model(torch.from_numpy(X).float()).detach().numpy().argmax(axis=1)
  prediction = labelencoder.inverse_transform(prediction)

  model_id = "CompVis/stable-diffusion-v1-4"
  device = "cuda"

  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
  pipe = pipe.to(device)

  for i in range(len(prediction)):
      prompt = prePrompt + prediction[i] + 'feelings'
      image = pipe(prompt).images[0]
      imageFile = 'predicted_image_'+str(idx[i])+prediction[i]+'.png'
      image.save(imageFile)


generate_image(EEGConformer(2524, 3), 'model/Conformer_EEG.pth', 'data/emotions.csv', range(2,3), 'an animated character going through')